In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    Aa = G + Noise*random.random((M,N)) # m轮收益总矩阵：包含每一轮的收益M*N
    return [A,Aa]

In [3]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [4]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 5
W = np.loadtxt("ER.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [5]:
print(Noise*random.random((M,T)))

[[0.681487   1.20326564 4.89165635 1.07076296 4.49323416 2.49156494
  1.48091186 1.43632425 2.23784116 1.7839017  3.64168004 3.83741699
  1.04186373 4.83878563 3.11101373 0.22786344 4.40003847 3.11673068
  2.1384704  3.40765058 1.23175884 4.44582622 0.54995869 3.28397975
  3.75462363 3.11530824 2.39343522 0.66419048 4.98305549 3.73872666
  2.44874463 0.37524703 0.8338022  3.49972682 2.55984609 0.06434849
  3.34828289 2.35424758 1.37118066 3.49949608]
 [1.49514946 0.56413995 3.93349596 1.68465678 3.82629017 2.2198006
  2.26904405 4.13404045 3.45249522 3.86514593 1.93464513 4.49526749
  1.08244897 4.60956914 2.68551457 3.7449877  4.916005   1.95905904
  3.25039733 0.48230999 0.87823516 3.32915228 3.98236859 0.40769645
  2.64957147 4.24744759 4.69517863 4.53962514 3.9399584  4.3306849
  4.72484089 4.71827546 0.33670258 2.78077988 3.11977476 3.09017523
  4.82027627 3.44343841 1.82179695 2.46147573]
 [0.51494555 2.22870726 1.97713581 1.4832022  3.84419022 0.80180403
  0.93337083 1.35366045 

In [11]:
X=zeros((T,T))
for j in range(T):
    w = TrainSTRidge(AA[j], y[:,j].reshape(-1,1),10**2,1)
    for i in range(w.shape[0]):
        X[i,j]=w[i,0]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

<ipython-input-3-13ac3e2d7f51>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-3-13ac3e2d7f51>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-11-0f2aa95661a7>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]


In [12]:
def TrainSTRidge_SPL(R, Ut, v, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge_SPL.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge_SPL(R,Ut,v,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge_SPL(X0, y, v, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
    else: w = np.linalg.lstsq(X*v,y*v)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]*v) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y*v))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [22]:
X=zeros((T,T))
v = ones((SV*M,1))
v1 = zeros((SV*M,1))
Los = zeros((SV*M,T))
losss = zeros((SV*M,1))
lambda_0=2
t=0
while (t<30):  
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v,10**2,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        for i in range(SV*M):
            losss[i]=np.mean(Los[i,:])
        t = t+1
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(losss)      
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

<ipython-input-12-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-12-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-22-5346f0e2178d>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]
<ipython-input-12-5e04d6e95ebc>:106: DeprecationWarning: elementwise comparison failed; this will raise an err

In [23]:
v

array([[0.99868311],
       [0.99848745],
       [0.99804135],
       [0.99845862],
       [0.99849062],
       [0.99824266],
       [0.99804631],
       [0.99826864],
       [0.99831466],
       [0.99804977],
       [0.99869315],
       [0.99867896],
       [0.9988304 ],
       [0.99890624],
       [0.99896009],
       [0.99871795],
       [0.99870427],
       [0.99873978],
       [0.99886688],
       [0.99893247],
       [0.99868255],
       [0.99870671],
       [0.99858693],
       [0.9985291 ],
       [0.99854637],
       [0.99829401],
       [0.99816122],
       [0.99829277],
       [0.99828893],
       [0.99811971],
       [0.99883226],
       [0.99888524],
       [0.99879034],
       [0.99886912],
       [0.99872797],
       [0.99862669],
       [0.99879409],
       [0.99866863],
       [0.99885913],
       [0.99882647],
       [0.99879473],
       [0.99829137],
       [0.9982494 ],
       [0.99812035],
       [0.99834625],
       [0.99827703],
       [0.99809093],
       [0.998

In [27]:
X=zeros((T,T))
v = ones((SV*M,T))
v1 = zeros((SV*M,T))
Los = zeros((SV*M,T))
lambda_0=2
t=0
while (t<30):
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v[:,j].reshape(-1,1),10**2,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        t = t+1
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

<ipython-input-12-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-12-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-27-196d2b896114>:17: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]
<ipython-input-12-5e04d6e95ebc>:106: DeprecationWarning: elementwise comparison failed; this will raise an err

In [28]:
v

array([[0.99690268, 0.99932899, 0.99962313, ..., 0.99937789, 0.99709882,
        0.99936387],
       [0.9983163 , 0.99886318, 0.99980639, ..., 0.99869197, 0.99690837,
        0.99997211],
       [0.99736186, 0.99767974, 0.9989742 , ..., 0.99969805, 0.99703282,
        0.99891204],
       ...,
       [0.99945531, 0.99687768, 0.99900857, ..., 0.99737693, 0.99802354,
        0.99852656],
       [0.99932383, 0.99902089, 0.99619765, ..., 0.99849552, 0.99935326,
        0.99741349],
       [0.99782077, 0.99893838, 0.99634069, ..., 0.99633437, 0.99904167,
        0.9994058 ]])